# Import

In [61]:
import sys
import os
# 標準エラー出力を/dev/nullにリダイレクト
sys.stderr = open(os.devnull, 'w')

import pandas as pd
import numpy as np
import japanize_matplotlib
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from geopy.distance import geodesic
import math as mt
import folium
import ast
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KernelDensity
from folium.plugins import HeatMap
import ast

# データの読み込み

In [2]:
gad_df = pd.read_csv(r"C:\Users\chaki\Desktop\jupyter_file\data\マイクロモビリティ\data\GAD_sapporo_Nov_weekend.csv")

In [78]:
mesh_df = pd.read_csv(r"C:\Users\chaki\Desktop\jupyter_file\data\マイクロモビリティ\data\mesh_sapporo.csv")
mesh_df['trip'] = 0
mesh_df['mesh'] = mesh_df['mesh'].apply(ast.literal_eval)

In [3]:
df = gad_df.copy()
def grid3rd(lon, lat):  # 3次メッシュ(8桁) 8分割x10分割=80分割
    return (int(mt.floor(lat*120      / 80)) * 1000000 + int(mt.floor((lon-100))             ) * 10000 +  # 1次メッシュ
            int(mt.floor(lat*120 % 80 / 10)) * 1000    + int(mt.floor((lon-100)*80 % 80 / 10)) * 100 +    # 2次メッシュ
            int(mt.floor(lat*120 % 10))      * 10      + int(mt.floor((lon-100)*80)) % 10               ) 
mesho_list = []
for i in range(len(df)):
    mesh = grid3rd(df['start_lon'][i],df['start_lat'][i])
    mesho_list.append(mesh)
df['mesh_o'] = mesho_list

meshd_list = []
for i in range(len(df)):
    mesh = grid3rd(df['end_lon'][i],df['end_lat'][i])
    meshd_list.append(mesh)
df['mesh_d'] = meshd_list
gad_df = df.copy()

In [4]:
gad_df = gad_df.rename(columns={"end_activity_type":"purpose"})
gad_df = gad_df.rename(columns={"longest_distance_mode":"mode"})
gad_df['trav_time_timedelta'] = pd.to_timedelta(gad_df['trav_time'])
# timedeltaから分のみ（総分数）を取得
gad_df['trav_time_minutes'] = round(gad_df['trav_time_timedelta'].dt.total_seconds() / 60)
gad_df = gad_df.dropna(subset=['trav_time'])
gad_df = gad_df.drop(['dep_time_tz', 'arr_time_tz', 'next_dep_time_tz', 'wait_time', 'euclidean_distance', 'start_activity_type_id', 'end_activity_type_id', 'start_key_code', 'end_key_code', 'modes_id', 'work_lon', 'work_lat', '短縮名称'], axis=1)


In [5]:
gad_df['judge'] = (gad_df['mode'] == 'car') & (gad_df['trav_time_minutes'] <= 5) & (gad_df['traveled_distance'] <= 2000) | (gad_df['mode'] == 'walk') & (gad_df['traveled_distance'] >= 1200)

# 全体代替可能トリップチェーン抽出

In [6]:
gad_walk = gad_df[gad_df['mode'].isin(['walk'])].reset_index(drop=True)
gad_car = gad_df[gad_df['mode'].isin(['car'])].reset_index(drop=True)
short_car =  gad_car[(gad_car['trav_time_minutes'] <= 5) & (gad_car['traveled_distance'] <= 2000)].reset_index(drop=True)
long_walk = gad_walk[gad_walk['traveled_distance'] >= 1200].reset_index(drop=True)
agent_df = pd.concat([short_car, long_walk], ignore_index=True)

In [7]:
# 代替可能性があるトリップのユーザーID
id_list = list(set(agent_df['agent_id']))

In [9]:
def check_all_rows_valid(df):
    return ((df['judge'] == True) | ((df['judge'] == False) & (df['mode'] == 'walk'))).all()

potential_true_df = pd.DataFrame(columns=gad_df.columns)
potential_false_df = pd.DataFrame(columns=gad_df.columns)
for i in range(len(id_list)):
    df = gad_df[gad_df['agent_id'].isin([id_list[i]])]
    if check_all_rows_valid(df):
        potential_true_df = pd.concat([potential_true_df, df], ignore_index=True)
    else:
        potential_false_df = pd.concat([potential_false_df, df], ignore_index=True)  


In [13]:
potential_true_df.to_csv(r'C:\Users\chaki\Desktop\jupyter_file\data\マイクロモビリティ\data\休日全体代替可能data.csv')

# 分析 

In [6]:
potential_df = pd.read_csv(r"C:\Users\chaki\Desktop\jupyter_file\data\マイクロモビリティ\data\休日全体代替可能data.csv", index_col=0)

In [ ]:
#potential_df = potential_true_df

In [9]:
true_df = gad_df[gad_df['judge'] == True].reset_index(drop=True)

## モード×目的のクロス集計表（トリップ単位）


In [10]:
purpose_list = list(true_df['purpose'].unique())
x_table = pd.DataFrame(columns=purpose_list,index = ['walk','car'])
y_table = pd.DataFrame(columns=purpose_list,index = purpose_list)

In [13]:
df = true_df.copy()

In [14]:
for i in range(len(purpose_list)):
    for j in ['walk','car']:
        x_table.loc[j, purpose_list[i]] = len(df[(df['purpose'] == purpose_list[i]) & (df['mode'] == j)])

In [15]:
for i in range(len(purpose_list)):
    for j in purpose_list:
        y_table.loc[j, purpose_list[i]] = len(df[(df['start_activity_type'] == purpose_list[i]) & (df['purpose'] == j)])

In [16]:
x_table

,others,eat,home,entertainment,retail,outdoor,grocery,medical,work
walk,31252,14493,26850,959,10852,1378,1806,1589,2222
car,164298,71738,214585,10980,80008,22584,18461,19278,8236


In [17]:
y_table

,others,eat,home,entertainment,retail,outdoor,grocery,medical,work
others,0,28518,112512,3510,30005,6277,5953,6316,2459
eat,26983,0,38406,1756,11850,1953,2223,2230,830
home,110532,42279,0,5395,45231,11851,10046,9320,6781
entertainment,3413,1529,4535,0,1400,372,279,307,104
retail,27326,11443,42217,1473,0,2462,2534,2585,820
outdoor,6416,2185,10941,372,2672,0,499,695,182
grocery,5354,2147,8922,263,2460,437,0,504,180
medical,5512,2061,9264,263,2529,529,512,0,197
work,1880,541,6814,65,660,182,122,194,0


## 移動手段×目的別分類

In [18]:
true_df = gad_df[gad_df['judge'] == True].reset_index(drop=True)

In [19]:
df = true_df.copy()
def grid3rd(lon, lat):  # 3次メッシュ(8桁) 8分割x10分割=80分割
    return (int(mt.floor(lat*120      / 80)) * 1000000 + int(mt.floor((lon-100))             ) * 10000 +  # 1次メッシュ
            int(mt.floor(lat*120 % 80 / 10)) * 1000    + int(mt.floor((lon-100)*80 % 80 / 10)) * 100 +    # 2次メッシュ
            int(mt.floor(lat*120 % 10))      * 10      + int(mt.floor((lon-100)*80)) % 10               ) 
mesho_list = []
for i in range(len(df)):
    mesh = grid3rd(df['start_lon'][i],df['start_lat'][i])
    mesho_list.append(mesh)
df['mesh_o'] = mesho_list

meshd_list = []
for i in range(len(df)):
    mesh = grid3rd(df['end_lon'][i],df['end_lat'][i])
    meshd_list.append(mesh)
df['mesh_d'] = meshd_list
true_df = df.copy()

In [20]:
walk_true = true_df[true_df['mode'] == 'walk'].reset_index(drop=True)
car_true = true_df[true_df['mode'] == 'car'].reset_index(drop=True)

In [26]:
walk_others = walk_true[walk_true["purpose"] == "others"]
walk_eat = walk_true[walk_true["purpose"] == "eat"]
walk_home = walk_true[walk_true["purpose"] == "home"]
walk_entertainment = walk_true[walk_true["purpose"] == "entertainment"]
walk_retail = walk_true[walk_true["purpose"] == "retail"]
walk_outdoor = walk_true[walk_true["purpose"] == "outdoor"]
walk_grocery = walk_true[walk_true["purpose"] == "grocery"]
walk_medical = walk_true[walk_true["purpose"] == "medical"]
walk_work = walk_true[walk_true["purpose"] == "work"]
car_others = car_true[car_true["purpose"] == "others"]
car_eat = car_true[car_true["purpose"] == "eat"]
car_home = car_true[car_true["purpose"] == "home"]
car_entertainment = car_true[car_true["purpose"] == "entertainment"]
car_retail = car_true[car_true["purpose"] == "retail"]
car_outdoor = car_true[car_true["purpose"] == "outdoor"]
car_grocery = car_true[car_true["purpose"] == "grocery"]
car_medical = car_true[car_true["purpose"] == "medical"]
car_work = car_true[car_true["purpose"] == "work"]


## カーネル密度推定

In [45]:
df_list = [walk_others,
 walk_eat,
 walk_home,
 walk_entertainment,
 walk_retail,
 walk_outdoor,
 walk_grocery,
 walk_medical,
 walk_work,
 car_others,
 car_eat,
 car_home,
 car_entertainment,
 car_retail,
 car_outdoor,
 car_grocery,
 car_medical,
 car_work]

In [ ]:
name_list = []
modes_list = ["walk","car"]
for i in range(len(modes_list)):
    for j in range(len(purpose_list)):
        name = modes_list[i] + "_" + purpose_list[j]
        name_list.append(name)


In [42]:
def mesh_to_latlon(mesh_code):
    """
    3次メッシュコード（1kmメッシュ）から中心の緯度経度を計算します。
    """
    mesh_code = str(mesh_code)
    if len(mesh_code) != 8:
        raise ValueError("メッシュコードは8桁である必要があります。")

    # 1次メッシュ
    lat1_val = int(mesh_code[0:2])
    lon1_val = int(mesh_code[2:4])
    
    # 2次メッシュ
    lat2_val = int(mesh_code[4])
    lon2_val = int(mesh_code[5])

    # 3次メッシュ
    lat3_val = int(mesh_code[6])
    lon3_val = int(mesh_code[7])

    # 左下の緯度経度を計算 (南西端)
    lat_sw = lat1_val * 2/3 + lat2_val * 1/12 + lat3_val * 1/120
    lon_sw = lon1_val + 100 + lon2_val * 1/8 + lon3_val * 1/80

    # 中心の緯度経度を計算
    # 3次メッシュの大きさは緯度方向に30秒、経度方向に45秒
    lat_center = lat_sw + 30/3600 / 2
    lon_center = lon_sw + 45/3600 / 2
    
    return lat_center, lon_center

In [52]:
for i in range(len(df_list)):
    df = df_list[i].copy()

    try:
        # 'mesh_o'列の各メッシュコードを緯度経度に変換
        df[['lat', 'lon']] = df['mesh_o'].apply(lambda x: pd.Series(mesh_to_latlon(x)))

        # カーネル密度推定の実行
        locations = df[['lat', 'lon']].values

        # bandwidth はデータの分布に応じて調整が必要です。
        # 1kmメッシュの場合、緯度経度の差が約0.01度になるため、それに近い値を設定すると良いでしょう。
        kde = KernelDensity(kernel='gaussian', bandwidth=0.01).fit(locations)

        # 地図の中心を計算
        map_center = [df['lat'].mean(), df['lon'].mean()]

        # Foliumで地図を作成
        m = folium.Map(location=map_center, zoom_start=12)

        # ヒートマップ用のグリッドデータを作成
        lat_min, lat_max = df['lat'].min() - 0.05, df['lat'].max() + 0.05
        lon_min, lon_max = df['lon'].min() - 0.05, df['lon'].max() + 0.05

        # グリッドの数を増やすと滑らかになりますが、計算時間が増加します。
        lat_grid = np.linspace(lat_min, lat_max, 100)
        lon_grid = np.linspace(lon_min, lon_max, 100)
        
        # 経度、緯度の順でグリッドを作成
        xx, yy = np.meshgrid(lon_grid, lat_grid)
        grid_points = np.vstack([yy.ravel(), xx.ravel()]).T

        # 各グリッドポイントでの密度を計算 (score_samplesは対数密度を返す)
        log_density = kde.score_samples(grid_points)
        density = np.exp(log_density)

        # ヒートマップデータを整形
        heat_data = [[point[0], point[1], weight] for point, weight in zip(grid_points, density)]
        
        # ヒートマップレイヤーを地図に追加
        HeatMap(heat_data).add_to(m)

        # 地図をHTMLファイルとして保存
        extension = ".html"
        full_file_name = f"{name_list[i]}{extension}"
        directory_path = r"C:\Users\chaki\Desktop\jupyter_file\data\マイクロモビリティ\カーネル密度推定"
        output_file = os.path.join(directory_path, full_file_name)
        m.save(output_file)
        
        print(f"✅ カーネル密度推定の結果を '{output_file}' に保存しました。")
        print("このHTMLファイルをブラウザで開いて結果を確認してください。")

    except NameError:
        print("⚠️ データフレーム 'df' が定義されていません。")
        print("お手持ちのデータを 'df' という名前のデータフレームに読み込むか、")
        print("プログラム冒頭のサンプルデータフレーム作成部分のコメントアウトを解除して実行してください。")
    except KeyError:
        print("⚠️ データフレーム 'df' に 'mesh_o' 列が存在しません。メッシュコードが格納されている列名を確認してください。")
    except Exception as e:
        print(f"エラーが発生しました: {e}")


✅ カーネル密度推定の結果を 'C:\Users\chaki\Desktop\jupyter_file\data\マイクロモビリティ\カーネル密度推定\walk_others.html' に保存しました。
このHTMLファイルをブラウザで開いて結果を確認してください。
✅ カーネル密度推定の結果を 'C:\Users\chaki\Desktop\jupyter_file\data\マイクロモビリティ\カーネル密度推定\walk_eat.html' に保存しました。
このHTMLファイルをブラウザで開いて結果を確認してください。
✅ カーネル密度推定の結果を 'C:\Users\chaki\Desktop\jupyter_file\data\マイクロモビリティ\カーネル密度推定\walk_home.html' に保存しました。
このHTMLファイルをブラウザで開いて結果を確認してください。
✅ カーネル密度推定の結果を 'C:\Users\chaki\Desktop\jupyter_file\data\マイクロモビリティ\カーネル密度推定\walk_entertainment.html' に保存しました。
このHTMLファイルをブラウザで開いて結果を確認してください。
✅ カーネル密度推定の結果を 'C:\Users\chaki\Desktop\jupyter_file\data\マイクロモビリティ\カーネル密度推定\walk_retail.html' に保存しました。
このHTMLファイルをブラウザで開いて結果を確認してください。
✅ カーネル密度推定の結果を 'C:\Users\chaki\Desktop\jupyter_file\data\マイクロモビリティ\カーネル密度推定\walk_outdoor.html' に保存しました。
このHTMLファイルをブラウザで開いて結果を確認してください。
✅ カーネル密度推定の結果を 'C:\Users\chaki\Desktop\jupyter_file\data\マイクロモビリティ\カーネル密度推定\walk_grocery.html' に保存しました。
このHTMLファイルをブラウザで開いて結果を確認してください。
✅ カーネル密度推定の結果を 'C:\Users\chaki\Desktop\jupyter_file

## 移動量割合

In [57]:
mesh_df

,code,mesh,trip
0,64403789,"[[42.983333333333334, 140.9875], [42.983333333...",0
1,64403799,"[[42.99166666666667, 140.9875], [42.9916666666...",0
2,64404709,"[[43.0, 140.9875], [43.0, 141.0], [43.00833333...",0
3,64404719,"[[43.00833333333333, 140.9875], [43.0083333333...",0
4,64404729,"[[43.016666666666666, 140.9875], [43.016666666...",0
...,...,...,...
1302,64416305,"[[43.166666666666664, 141.4375], [43.166666666...",0
1303,64416311,"[[43.175, 141.3875], [43.175, 141.399999999999...",0
1304,64416312,"[[43.175, 141.4], [43.175, 141.4125], [43.1833...",0
1305,64416321,"[[43.18333333333333, 141.3875], [43.1833333333...",0


In [85]:
for k in range(len(df_list)):
    df = df_list[k].copy()
    mesh_df0 = mesh_df.copy()
    count_df = df['mesh_d'].value_counts().reset_index()
    for i in range(len(mesh_df0)):
        for j in range(len(count_df)):
            if mesh_df0['code'][i] == count_df['mesh_d'][j]:
                mesh_df0['trip'][i] = count_df['count'][j]

    m = folium.Map(location=[43.068661,141.350755], zoom_start=11)  # 東京駅付近
    max_value = mesh_df['trip'].max()
    layer_1 = folium.FeatureGroup('sapporo').add_to(m)
    # layer_1(df_sapporo)
    for index, row in mesh_df0.iterrows():
        opacity = row['trip'] / mesh_df0['trip'].max()
        
        folium.Polygon(locations=row['mesh'], color='white1', fill=True, fill_color='blue', fill_opacity=opacity,tooltip=row['trip']).add_to(layer_1)
    layer_1.add_to(m)
    folium.LayerControl().add_to(m)
    extension = ".html"
    full_file_name = f"{name_list[k]}{extension}"
    directory_path = r"C:\Users\chaki\Desktop\jupyter_file\data\マイクロモビリティ\ヒートマップ\移動目的別origin"
    directory_path = r"C:\Users\chaki\Desktop\jupyter_file\data\マイクロモビリティ\ヒートマップ\移動目的destination"
    output_file = os.path.join(directory_path, full_file_name)
    m.save(output_file)
    print(f"✅'{output_file}' に保存しました。")
    

✅'C:\Users\chaki\Desktop\jupyter_file\data\マイクロモビリティ\ヒートマップ\移動目的destination\walk_others.html' に保存しました。
✅'C:\Users\chaki\Desktop\jupyter_file\data\マイクロモビリティ\ヒートマップ\移動目的destination\walk_eat.html' に保存しました。
✅'C:\Users\chaki\Desktop\jupyter_file\data\マイクロモビリティ\ヒートマップ\移動目的destination\walk_home.html' に保存しました。
✅'C:\Users\chaki\Desktop\jupyter_file\data\マイクロモビリティ\ヒートマップ\移動目的destination\walk_entertainment.html' に保存しました。
✅'C:\Users\chaki\Desktop\jupyter_file\data\マイクロモビリティ\ヒートマップ\移動目的destination\walk_retail.html' に保存しました。
✅'C:\Users\chaki\Desktop\jupyter_file\data\マイクロモビリティ\ヒートマップ\移動目的destination\walk_outdoor.html' に保存しました。
✅'C:\Users\chaki\Desktop\jupyter_file\data\マイクロモビリティ\ヒートマップ\移動目的destination\walk_grocery.html' に保存しました。
✅'C:\Users\chaki\Desktop\jupyter_file\data\マイクロモビリティ\ヒートマップ\移動目的destination\walk_medical.html' に保存しました。
✅'C:\Users\chaki\Desktop\jupyter_file\data\マイクロモビリティ\ヒートマップ\移動目的destination\walk_work.html' に保存しました。
✅'C:\Users\chaki\Desktop\jupyter_file\data\マイクロモビリティ\ヒートマップ\移動目的destin

In [86]:
m